<a href="https://colab.research.google.com/github/Stella-Achar-Oiro/NLP-Sentinel-Analysis/blob/main/Stella_Achar_Oiro_NLP_Sentinel_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install datasets transformers huggingface_hub -q

# Import key libraries and packages
import numpy as np
import os
import pandas as pd

from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TrainingArguments, Trainer
from google.colab import files
from google.colab import drive

# Disable Weights & Biases
os.environ["WANDB_DISABLED"] = "true"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
device_list = tf.test.gpu_device_name
print("Tensorflow version:", tf.__version__)

if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
device_list()

Tensorflow version: 2.12.0
GPU is available


'/device:GPU:0'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Load the datasets

url = 'https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_P5-NLP/master/zindi_challenge/data/SampleSubmission.csv'

sample_df = pd.read_csv(url)

url = 'https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_P5-NLP/master/zindi_challenge/data/Test.csv'

test_df = pd.read_csv(url).fillna("")

url = 'https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_P5-NLP/master/zindi_challenge/data/Train.csv'

train_df = pd.read_csv(url).dropna(axis = 0)

train_df.head()

test_df.head()

train_df.isnull().sum()

test_df.isnull().sum()

tweet_id     0
safe_text    0
dtype: int64

In [10]:
"""Fine-tuning the roberta model"""

train_df, eval = train_test_split(train_df, test_size=0.2, random_state=42, stratify= train_df['label'])

print(f"new dataframe shapes: train is {train_df.shape}, eval is {eval.shape}")


new dataframe shapes: train is (7999, 4), eval is (2000, 4)


In [11]:
# Save splitted subsets
train_df.to_csv("/content/drive/MyDrive/P5/train_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/P5/eval_subset.csv", index=False)


In [12]:
dataset = load_dataset('csv',
                        data_files={'train': '/content/drive/MyDrive/P5/train_subset.csv',
                        'eval': '/content/drive/MyDrive/P5/eval_subset.csv'}, encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0d885f1bb145989f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# Instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels=3)

# Define helper functions
## Function to transform labels
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}

## Function to tokenize data
def tokenize_data(example):
    return tokenizer(example['safe_text'], padding='max_length',truncation=True, max_length = 256)

# Tokenize the tweets
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and limit the columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

# Define training arguments
training_args = TrainingArguments(
    "covid_tweets_sentiment_analysis_model", 
    num_train_epochs=4,
    load_best_model_at_end=True,
    evaluation_strategy="epoch",
    save_strategy="epoch"
    )



Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
# Load the pretrained model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels=3)

# Define evaluation metrics
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Instantiate the training and evaluation sets
train_dataset = dataset["train"].shuffle(seed=24) 
eval_dataset = dataset["eval"].shuffle(seed=24)

#converting training data to PyTorch tensors to speed up training and adding padding:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Instantiate the trainer
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset,compute_metrics=compute_metrics)
trainer.train()

# Reinstantiate the trainer for evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Launch the final evaluation 
trainer.evaluate()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.630800,0.577171,0.775000
2,0.484200,0.619133,0.798000
3,0.359200,0.793091,0.788500
4,0.216600,1.088128,0.793000


{'eval_loss': 0.5771706700325012,
 'eval_accuracy': 0.775,
 'eval_runtime': 29.7351,
 'eval_samples_per_second': 67.261,
 'eval_steps_per_second': 8.408}

In [18]:
# Login to HF hub
notebook_login()


Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [19]:

# Push model and tokenizer to HF Hub
model.push_to_hub("NLP-Sentinel-Analysis")
tokenizer.push_to_hub("NLP-Sentinel-Analysis")


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Achar/NLP-Sentinel-Analysis/commit/26ab12f9acf36a0de4e4060d8b6c15e5ba367498', commit_message='Upload tokenizer', commit_description='', oid='26ab12f9acf36a0de4e4060d8b6c15e5ba367498', pr_url=None, pr_revision=None, pr_num=None)